<a href="https://colab.research.google.com/github/AshwinChowdary/CAPSTONE-PROJECT/blob/main/CAPSTONE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway panel bokeh ipywidgets ipywidgets_bokeh scikit-learn --quiet


In [2]:
import pandas as pd
import numpy as np
import pathway as pw
import panel as pn
import bokeh.plotting
import datetime
import joblib
from math import radians, sin, cos, sqrt, atan2
import contextlib
import logging

logging.getLogger("pathway_engine").setLevel(logging.CRITICAL)
pn.extension()


In [3]:
df = pd.read_csv("/content/dataset.csv")

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['VehicleEncoded'] = df['VehicleType'].map({'bike': 0.5, 'car': 1.0, 'truck': 1.5}).fillna(1.0)
df['TrafficEncoded'] = df['TrafficConditionNearby'].map({'low': 1, 'medium': 2, 'high': 3}).fillna(2)
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek

df = df.sort_values("Timestamp").reset_index(drop=True)

# Save for streaming
stream_df = df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity", "QueueLength", "IsSpecialDay",
                "VehicleEncoded", "TrafficEncoded", "Hour", "DayOfWeek"]]
stream_df.to_csv("parking_stream_full.csv", index=False)

# Save lot coordinates
coord_df = df[["SystemCodeNumber", "Latitude", "Longitude"]].drop_duplicates()
coord_map = coord_df.set_index("SystemCodeNumber")[["Latitude", "Longitude"]].to_dict("index")


In [4]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: float
    Capacity: float
    QueueLength: float
    IsSpecialDay: float
    VehicleEncoded: float
    TrafficEncoded: float
    Hour: int
    DayOfWeek: int


In [5]:
data = pw.demo.replay_csv("parking_stream_full.csv", schema=ParkingSchema, input_rate=500)
data = data.with_columns(t=pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"))


In [6]:
model = joblib.load("occupancy_predictor.pkl")

@pw.udf
def predict_demand(q, traffic, vtype, special, hour, dow):
    X = pd.DataFrame([[q, traffic, special, vtype, hour, dow]],
                     columns=["QueueLength", "TrafficEncoded", "IsSpecialDay", "VehicleEncoded", "Hour", "DayOfWeek"])
    return float(model.predict(X)[0])

@pw.udf
def compute_price(score):
    try:
        base = 10 * (1 + 0.5 * float(score))
        return round(min(max(base, 5), 20), 2)
    except:
        return 10.0

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a)) * 1000

@pw.udf
def fetch_lat(lot_id): return coord_map.get(lot_id, {}).get("Latitude", 0.0)

@pw.udf
def fetch_lon(lot_id): return coord_map.get(lot_id, {}).get("Longitude", 0.0)

@pw.udf
def adjust_by_nearby(lot_id, lat, lon, raw_price):
    try:
        count = 0
        for other_id, coords in coord_map.items():
            if other_id != lot_id:
                dist = haversine(lat, lon, coords["Latitude"], coords["Longitude"])
                if dist < 500:
                    count += 1
        return raw_price - 0.5 if count >= 2 else raw_price
    except:
        return raw_price


In [7]:
intermediate = (
    data.select(
        SystemCodeNumber=pw.this.SystemCodeNumber,
        t=pw.this.t,
        q=pw.this.QueueLength,
        traffic=pw.this.TrafficEncoded,
        vtype=pw.this.VehicleEncoded,
        special=pw.this.IsSpecialDay,
        hour=pw.this.Hour,
        dow=pw.this.DayOfWeek,
        lat=fetch_lat(pw.this.SystemCodeNumber),
        lon=fetch_lon(pw.this.SystemCodeNumber)
    )
    .with_columns(
        predicted_demand=predict_demand(pw.this.q, pw.this.traffic, pw.this.vtype,
                                        pw.this.special, pw.this.hour, pw.this.dow)
    )
    .with_columns(
        raw_price=compute_price(pw.this.predicted_demand)
    )
)

final_price_stream = intermediate.with_columns(
    price=adjust_by_nearby(pw.this.SystemCodeNumber, pw.this.lat, pw.this.lon, pw.this.raw_price)
)


In [8]:
lot_ids = sorted(df["SystemCodeNumber"].unique().tolist())

def plot_func(lot_id):
    sel = final_price_stream.filter(pw.this.SystemCodeNumber == lot_id)
    def p(source):
        fig = bokeh.plotting.figure(
            title=f"Live Price Plot for Lot: {lot_id}",
            x_axis_type="datetime",
            width=800,
            height=400
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=6, color="red")
        return fig
    return sel.plot(p, sorting_col="t")

selector = pn.widgets.Select(name="Lot", options=lot_ids, value=lot_ids[0])
viz = pn.bind(plot_func, lot_id=selector)
pn.Column(selector, viz).servable()


Column
    [0] Select(name='Lot', options=['BHMBCCMKT01', ...], value='BHMBCCMKT01')
    [1] ParamFunction(function, _pane=Column, defer_load=False)

In [ ]:
with contextlib.redirect_stdout(None):
    pw.run()
